In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.signal import butter, filtfilt
from scipy.signal import czt
import matplotlib.pyplot as plt


# Feature Extraction from POS (rPPG) Signal

This notebook extracts standard physiological features from the BVP/POS signal, which are typically used for **arousal classification**.

We will compute:

### 🔹 Time-domain features
- **Mean** — average blood volume
- **Standard deviation** — variability of the signal
- **RMSSD** — root mean square of successive differences (HRV proxy)
- **SDSD** — std of successive differences
- **Skewness** — asymmetry of the distribution
- **Kurtosis** — tailedness of the distribution
- **pNN20** — % of differences > 20 units
- **pNN50** — % of differences > 50 units

### 🔹 Spectral-domain features (using CZT)
- **VLF power (0–0.1 Hz)**
- **LF power (0.1–0.3 Hz)**
- **HF power (0.3–0.5 Hz)**
- **LF/HF ratio** — arousal-related index
- **Peak frequency** → used to estimate HR

At the end, all features will be stored in a **feature vector**.

## Time Domain Features

These features measure variability and shape of the BVP signal itself.

### Mean
Average of the signal → general blood flow baseline.

### Standard Deviation
Measures how much the signal varies. Higher arousal often increases variability.

### RMSSD
Root Mean Square of Successive Differences.
A proxy for **HRV** — decreases when arousal increases.

### SDSD
Standard deviation of the successive differences.

### Skewness
Tells whether the signal leans left or right.

### Kurtosis
Tells how heavy the tails of the distribution are.

### pNN20 / pNN50
Proportion of successive differences greater than 20 or 50 units.
Higher values = higher autonomic activity.

In [ ]:
def extract_time_features(bvp):
    diff = np.diff(bvp)

    features = {
        "mean": np.mean(bvp),
        "std": np.std(bvp),
        "rmssd": np.sqrt(np.mean(diff**2)),
        "sdsd": np.std(diff),
        "skewness": skew(bvp),
        "kurtosis": kurtosis(bvp),
        "pnn20": np.mean(np.abs(diff) > 20),
        "pnn50": np.mean(np.abs(diff) > 50),
    }

    return features

# Example usage:
# time_feats = extract_time_features(bvp)
# time_feats


## Spectral Features (Using the Chirp Z-Transform)

We use CZT instead of FFT because it allows us to zoom into the exact
frequency range of interest.

Heart-related BVP frequencies are typically between **0.6–4 Hz** (36–240 bpm).

We extract:

### VLF (0.00–0.10 Hz)
Baseline drift. Not directly related to arousal.

### LF (0.10–0.30 Hz)
Sympathetic activity — increases with arousal.

### HF (0.30–0.50 Hz)
Parasympathetic activity — decreases with arousal.

### LF/HF Ratio
Classic autonomic balance measure.
Higher LF/HF = higher arousal / stress.

We also detect the **peak frequency**, from which HR is estimated.


In [ ]:
def extract_czt_features(bvp, fs=60, fmin=0, fmax=4, n_points=4096):

    # Compute CZT
    freq_resolution = (fmax - fmin) / n_points
    A = np.exp(1j * 2 * np.pi * fmin / fs)
    W = np.exp(-1j * 2 * np.pi * (fmax - fmin) / (n_points * fs))

    czt_output = czt(bvp, n_points, W, A)
    freqs = np.linspace(fmin, fmax, n_points)
    power = np.abs(czt_output)

    # Band power helper
    def bandpower(fmin, fmax):
        mask = (freqs >= fmin) & (freqs <= fmax)
        return np.trapz(power[mask], freqs[mask])

    vlf = bandpower(0.00, 0.10)
    lf  = bandpower(0.10, 0.30)
    hf  = bandpower(0.30, 0.50)
    lfhf = lf / hf if hf > 0 else 0

    # HR estimation
    peak_freq = freqs[np.argmax(power)]
    hr = peak_freq * 60

    return {
        "vlf": vlf,
        "lf": lf,
        "hf": hf,
        "lfhf": lfhf,
        "peak_freq": peak_freq,
        "hr": hr
    }

# Example:
# spec_feats = extract_czt_features(bvp)
# spec_feats


## Combine All Features Into a Single Feature Vector

Here we merge:

- Time-domain features  
- CZT spectral features  
- HR estimate  

into a final dictionary and a NumPy array.


In [ ]:
def extract_all_features(bvp, fs=60):
    time_feats = extract_time_features(bvp)
    spec_feats = extract_czt_features(bvp, fs)

    # Combine in a single dict
    features = {**time_feats, **spec_feats}

    # Also return a clean NumPy array ready for ML models
    feature_array = np.array(list(features.values()), dtype=float)

    return features, feature_array

# Example:
# features, feature_array = extract_all_features(bvp)
# features
# feature_array


## Visualize BVP and Spectrum

In [ ]:
# Plot the BVP signal
plt.figure(figsize=(12,4))
plt.plot(bvp)
plt.title("BVP Signal")
plt.xlabel("Samples")
plt.ylabel("Amplitude")
plt.grid(True)
plt.show()

# Plot the CZT spectrum
spec = extract_czt_features(bvp)
plt.figure(figsize=(12,4))
plt.plot(
    np.linspace(0, 4, 4096),
    np.abs(czt(bvp, 4096))
)
plt.title("CZT Spectrum")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.xlim(0, 4)
plt.grid(True)
plt.show()